# Class for controlling database

In [238]:
import sqlite3
from sqlite3 import OperationalError
import pandas as pd

class DBClient:
    def __init__(self, db_path):
        '''Simple interface for creating a game database'''
        self.conn = sqlite3.connect(db_path)
        self.cursor = self.conn.cursor()

    def create_table(self):
        try:
            self.cursor.execute('''CREATE TABLE games 
                (id integer primary key autoincrement, Name, RawgID, 
                SteamURL, Metacritic, Genres, Indie,
                Presence, MonetizationModel, Platform, Graphics, Storage,
                RatingsBreakdown, ReleaseDate, Soundtrack, Franchise,
                OriginalCost, DiscountedCost, Players, Controller, Languages,
                ESRB, Achievements, Publisher, Description)''')
            self.conn.commit()
        except OperationalError as e:
            print(f'error creating table: {e}')

    def drop_table(self):
        self.cursor.execute('''DROP TABLE games''')
        self.conn.commit()

    def add_game(self, game):
        try:
            res = self.cursor.execute('''INSERT INTO games(
                    Name, RawgID, 
                    SteamURL, Metacritic, Genres, Indie,
                    Presence, MonetizationModel, Platform, Graphics, Storage,
                    RatingsBreakdown, ReleaseDate, Soundtrack, Franchise,
                    OriginalCost, DiscountedCost, Players, Controller, Languages,
                    ESRB, Achievements, Publisher, Description) VALUES(?,?,?,?,?,?,
                    ?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)''', tuple(game.values()))
            self.conn.commit()
        except:
            print(f'error adding game {game}')

    def get_game(self, gameId):
        t = (gameId,)
        self.cursor.execute('SELECT * FROM games WHERE id=?', t)
        res = self.cursor.fetchone()
        return res

    def get_all_games(self):
        self.cursor.execute('SELECT * FROM games')
        res = self.cursor.fetchall()
        return res

    def to_csv(self, path):
        temp_df = pd.read_sql_query('SELECT * FROM games', con=self.conn)
        temp_df.to_csv(path)

    def delete_game(self, gameId):
        try:
            t = (gameId, )
            self.cursor.execute('DELETE FROM games WHERE id=?', t)
            self.conn.commit()
            return True
        except:
            print(f'error deleting game {gameId}')
            return False
        
    def close(self):
        self.conn.close()
        

## Test

In [231]:
test_game = { 
    'Name': 'sample',
    'RawgID': 20, 
    'SteamURL': 'https://sample.com', 
    'Metacritic': 100, 
    'Genres': 'fps, platformer', 
    'Indie': True,
    'Presence': 83, 
    'MonetizationModel': 'Paid', 
    'Platform': 'Windows', 
    'Graphics': '4gb GPU', 
    'Storage': 180,
    'RatingsBreakdown': '34/45/15', 
    'ReleaseDate': 'January 14, 2020', 
    'Soundtrack': True, 
    'Franchise': True,
    'OriginalCost': 45, 
    'DiscountedCost': 22, 
    'Players': 4, 
    'Controller': False, 
    'Languages': 'English, Mandarin',
    'ESRB': 'Teen', 
    'Achievements': 55, 
    'Publisher': 'idSoftware', 
    'Description': 'lots of stuff'
}

In [232]:
tuple(test_game.values())

('sample',
 20,
 'https://sample.com',
 100,
 'fps, platformer',
 True,
 83,
 'Paid',
 'Windows',
 '4gb GPU',
 180,
 '34/45/15',
 'January 14, 2020',
 True,
 True,
 45,
 22,
 4,
 False,
 'English, Mandarin',
 'Teen',
 55,
 'idSoftware',
 'lots of stuff')

In [243]:
# setup table
dbc = DBClient('/content/sample.db')

# create table
dbc.create_table()

# add sample game
dbc.add_game(test_game)

# get one game
res = dbc.get_game(1)
print(f'game id 1: {res}')

# get all games
games = dbc.get_all_games()
print(f'all games: {games}')

# export as csv
dbc.to_csv('sample.csv')

# delete a game
result = dbc.delete_game(1)
print(f'game deleted: {result}')

# get all games
games = dbc.get_all_games()
print(f'all games: {games}')

# drop table
dbc.drop_table()

# close connection
dbc.close()

game id 1: (1, 'sample', 20, 'https://sample.com', 100, 'fps, platformer', 1, 83, 'Paid', 'Windows', '4gb GPU', 180, '34/45/15', 'January 14, 2020', 1, 1, 45, 22, 4, 0, 'English, Mandarin', 'Teen', 55, 'idSoftware', 'lots of stuff')
all games: [(1, 'sample', 20, 'https://sample.com', 100, 'fps, platformer', 1, 83, 'Paid', 'Windows', '4gb GPU', 180, '34/45/15', 'January 14, 2020', 1, 1, 45, 22, 4, 0, 'English, Mandarin', 'Teen', 55, 'idSoftware', 'lots of stuff')]
game deleted: True
all games: []


In [246]:
pd.read_csv('sample.csv', index_col=0)

,id,Name,RawgID,SteamURL,Metacritic,Genres,Indie,Presence,MonetizationModel,Platform,Graphics,Storage,RatingsBreakdown,ReleaseDate,Soundtrack,Franchise,OriginalCost,DiscountedCost,Players,Controller,Languages,ESRB,Achievements,Publisher,Description
0,1,sample,20,https://sample.com,100,"fps, platformer",1,83,Paid,Windows,4gb GPU,180,34/45/15,"January 14, 2020",1,1,45,22,4,0,"English, Mandarin",Teen,55,idSoftware,lots of stuff
